# NLP- Naturel Language Processing

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

!pip install textblob
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

import re
import nltk

from nltk import word_tokenize, pos_tag
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize
from nltk.corpus import wordnet

import warnings
warnings.simplefilter("ignore", UserWarning)
warnings.simplefilter("ignore", FutureWarning)
warnings.simplefilter("ignore", DeprecationWarning)

In [ ]:
df["Comment"]=df["Comment"].str.lower() #kucuk harfe cevir
df["Comment"]=df["Comment"].str.replace("[^\w\s]","") #noktalama işaretlerini kaldır
df["Comment"]=df["Comment"].str.replace("\d+","") #d digit rakamları kaldır
df["Comment"]=df["Comment"].str.replace("\n"," ") #Satır sonu,\n 
df["Comment"]=df["Comment"].str.replace("\r","") #enter a basılmışsa \r
df["Comment"]=df["Comment"].str.replace("\t","")
df["Comment"]=df["Comment"].str.strip() #satır başında boşlukları kaldır 

In [ ]:
df["Date"]=pd.to_datetime(df["Date"])

df["Day"]=df["Date"].dt.dayofweek
df["Month"]=df["Date"].dt.month
df["Year"]=df["Date"].dt.year


df['WeekDay']=df['Date'].dt.day_name()
df['Month_Name']=df['Date'].dt.month_name()

df.drop("Date",axis=1,inplace=True)
# Tarih donusumleri

In [ ]:
from textblob import TextBlob
df[['polarity','subjectivity']]=df['text'].apply(lambda t:pd.Series(TextBlob(t).sentiment))
# Kelimelerin polaritysini buluyoruz

In [ ]:
nltk.download("stopwords")
stop_word_list=nltk.corpus.stopwords.words("turkish")
# Gereksiz kelimeleri cikarma

In [ ]:
from wordcloud import WordCloud
def wc(data,bgcolor):
    from PIL import Image
    mask = np.array(Image.open("cloud.png"))
    wc=WordCloud(background_color=bgcolor, width=800, height=400,mask=mask,max_words=100).generate(' '.join(data))
    plt.figure( figsize=(20,10) )
    plt.imshow(wc)
    plt.axis('off')
    plt.show()

In [ ]:
wc(df["title"],'white')

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

In [ ]:
b=MultinomialNB()
model=b.fit(x_train_dtm,y_train)
b_predict=b.predict(x_test_dtm)

In [ ]:
accuracy_score(y_test,b_predict)

# Sentiment Analysis 

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [ ]:
df.loc[df['subjectivity']>0.5,'sentiment']="pozitive"
df.loc[df['subjectivity']==0.5,'sentiment']="neutral"
df.loc[df['subjectivity']<0.5,'sentiment']="negative"
df
# 0.05 ebsulon komsulugunda olanlara gore sinifladik

In [ ]:
x,y=df['Comment'],df['sentiment']

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=60)

In [ ]:
vect=CountVectorizer()
x_train_dtm=vect.fit_transform(x_train,y_train)
x_test_dtm=vect.transform(x_test)

In [ ]:
x_train_dtm.shape,x_test_dtm.shape

In [ ]:
def tokenize_test(vect,x,y):
    from sklearn.naive_bayes import GaussianNB
    from sklearn.naive_bayes import BernoulliNB
    from sklearn.svm import SVC
    from sklearn.neighbors import KNeighborsClassifier
    from sklearn.tree import DecisionTreeClassifier
    from sklearn.ensemble import RandomForestClassifier
    from sklearn.linear_model import LogisticRegression
    from xgboost import XGBClassifier #yüklemek için !pip install xgboost kullandım
    
    from sklearn.metrics import accuracy_score,precision_score,recall_score,f1_score
    from sklearn.metrics import confusion_matrix,classification_report
    
    x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=60)
    
    print(x_train.shape,x_test.shape,y_train.shape,y_test.shape)
    
    g=GaussianNB()
    b=BernoulliNB()
    SVC=SVC()
    KN=KNeighborsClassifier()
    D=DecisionTreeClassifier()
    R=RandomForestClassifier()
    Log=LogisticRegression()
    XGBC=XGBClassifier()
    
    algos=[g,b,SVC,KN,D,R,Log,XGBC]
    algo_names=['GaussianNB','BernoulliNB','SVC','KNeighborsClassifier','DecisionTreeClassifier','RandomForestClassifier','LogisticRegression','XGBClassifier']   
    
    accuracy_scored=[]    
        
    x_train_dtm=vect.fit_transform(x_train).toarray()
    x_test_dtm=vect.transform(x_test).toarray()
    
    for item in algos:
        item.fit(x_train_dtm,y_train)
        accuracy_scored.append(accuracy_score(y_test,item.predict(x_test_dtm)))
    result=pd.DataFrame(accuracy_scored,columns=['accuracy_score'],index=algo_names)
    result.accuracy_score=accuracy_scored
    return result.sort_values('accuracy_score',ascending=False)  

In [ ]:
vect=CountVectorizer()
tokenize_test(vect,x,y)